# Assignment 2

In this assigment, we will work with the *Adult* data set. Please download the data from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/2/adult). Extract the data files into the subdirectory: `../05_src/data/adult/` (relative to `./05_src/`).

# Load the data

Assuming that the files `adult.data` and `adult.test` are in `../05_src/data/adult/`, then you can use the code below to load them.

In [3]:
import pandas as pd
columns = [
    'age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status',
    'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week',
    'native-country', 'income'
]
adult_dt = (pd.read_csv('../../05_src/data/adult/adult.data', header = None, names = columns)
              .assign(income = lambda x: (x.income.str.strip() == '>50K')*1))


# Get X and Y

Create the features data frame and target data:

+ Create a dataframe `X` that holds the features (all columns that are not `income`).
+ Create a dataframe `Y` that holds the target data (`income`).
+ From `X` and `Y`, obtain the training and testing data sets:

    - Use a train-test split of 70-30%. 
    - Set the random state of the splitting function to 42.

In [4]:
from sklearn.model_selection import train_test_split

# Features and target variable definition
X = adult_dt.drop(columns=['income'])
y = adult_dt['income']

# Split data into training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, test_size = 0.3, random_state = 42)

## Random States

Please comment: 

+ What is the [random state](https://scikit-learn.org/stable/glossary.html#term-random_state) of the [splitting function](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)? 
+ Why is it [useful](https://en.wikipedia.org/wiki/Reproducibility)?

+ It's a seed value for the random number generator, allowing you to get the same split each time you run the code.

+ It is useful because it ensures that your results can be replicated. The random number generator produces the same sequence of numbers every time, leading to consistent results across runs. 

# Preprocessing

Create a [Column Transformer](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html) that treats the features as follows:

- Numerical variables

    * Apply [KNN-based imputation for completing missing values](https://scikit-learn.org/stable/modules/generated/sklearn.impute.KNNImputer.html):
        
        + Consider the 7 nearest neighbours.
        + Weight each neighbour by the inverse of its distance, causing closer neigbours to have more influence than more distant ones.
    * [Scale features using statistics that are robust to outliers](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.RobustScaler.html#sklearn.preprocessing.RobustScaler).

- Categorical variables: 
    
    * Apply a [simple imputation strategy](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html#sklearn.impute.SimpleImputer):

        + Use the most frequent value to complete missing values, also called the *mode*.

    * Apply [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html):
        
        + Handle unknown labels if they exist.
        + Drop one column for binary variables.
    
    
The column transformer should look like this:

![](./images/assignment_2__column_transformer.png)

In [6]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import RobustScaler, OneHotEncoder

# define numerical and categorical columns
num_cols = list(X.select_dtypes(include=[float, int]).columns)
cat_cols = list(X.select_dtypes(include=[object]).columns)

# KNN-based imputation for numerical variables
pipe_num = Pipeline([
    ('imputer', KNNImputer(n_neighbors = 7, weights = 'distance')),
    ('standardizer', RobustScaler())
])

# Simple imputation for categorical variables
pipe_cat = Pipeline([
    ('imputer', SimpleImputer(strategy = 'most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown = 'infrequent_if_exist', drop = 'if_binary'))
])

# Define columns transformer for numerical and categorical pipelines 
ctransform = ColumnTransformer([
    ('num_transforms', pipe_num, num_cols),
    ('cat_transforms', pipe_cat, cat_cols),
], remainder='drop')

## Model Pipeline

Create a [model pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html): 

+ Add a step labelled `preprocessing` and assign the Column Transformer from the previous section.
+ Add a step labelled `classifier` and assign a [`RandomForestClassifier()`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) to it.

The pipeline looks like this:

![](./images/assignment_2__pipeline.png)

In [7]:
from sklearn.ensemble import RandomForestClassifier

# Define pipeline for Random Forest Classifier
pipe_rf = Pipeline([
    ('preprocessing', ctransform),
    ('classifier', RandomForestClassifier())
])

# Cross-Validation

Evaluate the model pipeline using [`cross_validate()`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html):

+ Measure the following [preformance metrics](https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values): negative log loss, ROC AUC, accuracy, and balanced accuracy.
+ Report the training and validation results. 
+ Use five folds.


In [8]:
from sklearn.model_selection import cross_validate

# Define score performance metrics
scoring = ['neg_log_loss', 'roc_auc', 'accuracy', 'balanced_accuracy']

#Evaluate cross validation
res_rf_dict = cross_validate(pipe_rf, X_train, y_train, cv = 5, scoring = scoring, return_train_score = True)

c:\ProgramData\anaconda3\envs\dsi_participant\lib\site-packages\sklearn\preprocessing\_encoders.py:242: UserWarning: Found unknown categories in columns [7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
c:\ProgramData\anaconda3\envs\dsi_participant\lib\site-packages\sklearn\preprocessing\_encoders.py:242: UserWarning: Found unknown categories in columns [7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


Display the fold-level results as a pandas data frame and sorted by negative log loss of the test (validation) set.

In [9]:
# Convert results to DataFrame
res_rf = pd.DataFrame(res_rf_dict).sort_values(by = 'test_neg_log_loss')
res_rf

,fit_time,score_time,test_neg_log_loss,train_neg_log_loss,test_roc_auc,train_roc_auc,test_accuracy,train_accuracy,test_balanced_accuracy,train_balanced_accuracy
4,9.588170,0.170117,-0.386780,-0.081904,0.902133,1.0,0.857174,1.000000,0.776813,1.000000
1,10.576222,0.219122,-0.379513,-0.081556,0.902580,1.0,0.849309,0.999945,0.770235,0.999887
3,9.824971,0.183997,-0.365656,-0.082224,0.905820,1.0,0.858929,1.000000,0.780433,1.000000
0,7.721101,0.206103,-0.363414,-0.081316,0.904385,1.0,0.852599,0.999945,0.777938,0.999887
2,9.460607,0.262237,-0.354676,-0.081066,0.902619,1.0,0.852128,1.000000,0.772867,1.000000


Calculate the mean of each metric. 

In [10]:
res_rf.mean()

fit_time                   9.434214
score_time                 0.208315
test_neg_log_loss         -0.370008
train_neg_log_loss        -0.081613
test_roc_auc               0.903507
train_roc_auc              1.000000
test_accuracy              0.854028
train_accuracy             0.999978
test_balanced_accuracy     0.775658
train_balanced_accuracy    0.999955
dtype: float64

Calculate the same performance metrics (negative log loss, ROC AUC, accuracy, and balanced accuracy) using the testing data `X_test` and `Y_test`. Display results as a dictionary.

*Tip*: both, `roc_auc()` and `neg_log_loss()` will require prediction scores from `pipe.predict_proba()`. However, for `roc_auc()` you should only pass the last column `Y_pred_proba[:, 1]`. Use `Y_pred_proba` with `neg_log_loss()`.

In [12]:
from sklearn.metrics import log_loss, roc_auc_score, accuracy_score, balanced_accuracy_score

# Fit model training data
pipe_rf.fit(X_train, y_train)

# Predict probabilities and label test data
y_pred_test = pipe_rf.predict(X_test)
y_proba_test = pipe_rf.predict_proba(X_test)

# Calculate performance metrics
metrics_test = {
    'log_loss_test': log_loss(y_test, y_proba_test),
    'roc_auc_score_test': roc_auc_score(y_test, y_proba_test[:,1]),
    'accuracy_score_test': accuracy_score(y_test, y_pred_test),
    'balanced_accuracy_score_test': balanced_accuracy_score(y_test, y_pred_test),
}
metrics_test

{'log_loss_test': 0.39148159918419484,
 'roc_auc_score_test': 0.899546776481418,
 'accuracy_score_test': 0.8531067663015662,
 'balanced_accuracy_score_test': 0.7727787352174122}

# Target Recoding

In the first code chunk of this document, we loaded the data and immediately recoded the target variable `income`. Why is this [convenient](https://scikit-learn.org/stable/modules/model_evaluation.html#binary-case)?

The specific line was:

```
adult_dt = (pd.read_csv('../05_src/data/adult/adult.data', header = None, names = columns)
              .assign(income = lambda x: (x.income.str.strip() == '>50K')*1))
```

*Answer:*

Recoding the 'income' variable during data loading is convenient because it simplifies the subsequent analysis by converting income into a binary format (1 for >50K and 0 otherwise) at the start.

## Criteria

The [rubric](./assignment_2_rubric_clean.xlsx) contains the criteria for assessment.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-2`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_2.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.

# Reference

Becker,Barry and Kohavi,Ronny. (1996). Adult. UCI Machine Learning Repository. https://doi.org/10.24432/C5XW20.